In [19]:
from google.colab import files
uploaded = files.upload()

Saving dataset_fully_cleaned.xlsx to dataset_fully_cleaned (3).xlsx


In [20]:
import pandas as pd

df_merged = pd.read_excel("nba_cleaned.xlsx")

In [21]:
# Quick overview of the data
df_merged.info()
df_merged.describe()
df_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 53 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rk              430 non-null    int64  
 1   player          430 non-null    object 
 2   age             430 non-null    int64  
 3   pos             430 non-null    object 
 4   g               430 non-null    int64  
 5   gs              430 non-null    int64  
 6   mp              430 non-null    float64
 7   fg              430 non-null    float64
 8   fga             430 non-null    float64
 9   fg2             430 non-null    float64
 10  3p              430 non-null    float64
 11  3pa             430 non-null    float64
 12  3p3             415 non-null    float64
 13  2p              430 non-null    float64
 14  2pa             430 non-null    float64
 15  2p4             430 non-null    float64
 16  efg             430 non-null    float64
 17  ft              430 non-null    flo

,rk,player,age,pos,g,gs,mp,fg,fga,fg2,...,ows,dws,ws,ws48,obpm,dbpm,bpm,vorp,team,salary_2024_25
0,1,Joel Embiid,29,C,39,39,33.6,11.5,21.8,0.529,...,5.2,2.3,7.5,0.275,8.5,3.1,11.6,4.5,Phi,51415938
1,2,Luka Dončić,24,PG,70,70,37.5,11.5,23.6,0.487,...,8.5,3.5,12.0,0.220,8.3,1.7,9.9,8.0,Lal,43031940
2,3,Giannis Antetokounmpo,29,PF,73,73,35.2,11.5,18.8,0.611,...,9.5,3.7,13.2,0.246,6.7,2.4,9.0,7.2,Mil,48787676
3,4,Shai Gilgeous-Alexander,25,PG,75,75,34.0,10.6,19.8,0.535,...,10.5,4.2,14.6,0.275,6.7,2.3,9.0,7.1,Okc,35859950
4,5,Jalen Brunson,27,PG,77,77,35.4,10.3,21.4,0.479,...,8.8,2.4,11.2,0.198,6.3,-0.4,5.8,5.4,Nyk,24960001


In [22]:
# Custom Efficiency score to evaluate player impact

df_merged['efficiency'] = (
    (df_merged['pts'] + df_merged['trb'] + df_merged['ast'] - df_merged['tov'])
    / df_merged['g']
)

In [23]:
# Value for money: Efficiency/$1M

df_merged['efficiency_per_million'] = (
    df_merged['efficiency'] / (df_merged['salary_2024_25'] / 1_000_000)
)

In [24]:
# Reliable sample size: games played > 50

df_filtered = df_merged[df_merged['g'] > 50]

In [25]:
# K-means clustering: Group players based on role and value

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = df_filtered[['efficiency', 'usg', 'salary_2024_25']].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=42)
df_filtered['value_cluster'] = kmeans.fit_predict(X_scaled)

/tmp/ipython-input-25-3328543484.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['value_cluster'] = kmeans.fit_predict(X_scaled)


In [26]:
# Visualise player clusters: to see high-usage, low-efficiency players or low-paid high-efficiency players

import plotly.express as px

color_sequence = px.colors.qualitative.Set2

fig = px.scatter(
    df_filtered,
    x='usg',
    y='efficiency',
    color='value_cluster',
    hover_name='player',
    hover_data={
        'salary_2024_25': True,
        'g': True,
        'usg': ':.2f',
        'efficiency': ':.2f',
        'value_cluster': False
    },
    title="Interactive Player Clusters: Usage vs Efficiency (50+ games)",
    labels={
        'usg': 'Usage %',
        'efficiency': 'Efficiency',
        'value_cluster': 'Cluster'
    },
    color_discrete_sequence=color_sequence
)

fig.update_layout(height=600, width=900)
fig.show()

In [27]:
# Undervalued Players: Above median efficiency AND below median salary
# Concrete list of high-impact, low-cost players — ideal trade/retention targets

# Calculate medians
eff_median = df_filtered['efficiency'].median()
sal_median = df_filtered['salary_2024_25'].median()

# Flag under/over value
df_filtered['undervalued'] = (df_filtered['efficiency'] > eff_median) & (df_filtered['salary_2024_25'] < sal_median)
df_filtered['overvalued'] = (df_filtered['efficiency'] < eff_median) & (df_filtered['salary_2024_25'] > sal_median)

# Quick check
undervalued_players = df_filtered[df_filtered['undervalued']][['player', 'efficiency', 'salary_2024_25']]
print("✅ Undervalued Players Sample:\n", undervalued_players.head())

✅ Undervalued Players Sample:
                player  efficiency  salary_2024_25
34         Cam Thomas    0.404545         4041249
40     Alperen Şengün    0.520635         5424654
46  Russell Westbrook    0.272059         5631296
53       Franz Wagner    0.372222         7007092
61     Jalen Williams    0.364789         4775760


/tmp/ipython-input-27-3650980849.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-27-3650980849.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
import plotly.express as px

fig = px.scatter(
    df_filtered,
    x='salary_2024_25',
    y='efficiency',
    color=df_filtered['undervalued'].map({True: 'Undervalued', False: 'Not Undervalued'}),
    hover_name='player',
    title='NBA Player Efficiency vs Salary (Undervalued Players Highlighted)',
    labels={'salary_2024_25': 'Salary ($)', 'efficiency': 'Efficiency'},
)

fig.show()

In [29]:
df_filtered.head()

,rk,player,age,pos,g,gs,mp,fg,fga,fg2,...,dbpm,bpm,vorp,team,salary_2024_25,efficiency,efficiency_per_million,value_cluster,undervalued,overvalued
1,2,Luka Dončić,24,PG,70,70,37.5,11.5,23.6,0.487,...,1.7,9.9,8.0,Lal,43031940,0.698571,0.016234,2,False,False
2,3,Giannis Antetokounmpo,29,PF,73,73,35.2,11.5,18.8,0.611,...,2.4,9.0,7.2,Mil,48787676,0.616438,0.012635,2,False,False
3,4,Shai Gilgeous-Alexander,25,PG,75,75,34.0,10.6,19.8,0.535,...,2.3,9.0,7.1,Okc,35859950,0.528000,0.014724,2,False,False
4,5,Jalen Brunson,27,PG,77,77,35.4,10.3,21.4,0.479,...,-0.4,5.8,5.4,Nyk,24960001,0.475325,0.019043,2,False,False
5,6,Devin Booker,27,PG,68,68,36.0,9.4,19.2,0.492,...,-1.0,3.0,3.1,Pho,49205800,0.527941,0.010729,2,False,False


In [30]:
# Download final data for tableau

df_filtered.to_excel("dataset_final.xlsx", index=False)

from google.colab import files
files.download("dataset_final.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>